# Chi-Merge Test : Data Discretization for Numeric Features

In [35]:
#Import required python libraries 
import pandas as pd
import numpy as np

In [36]:
#Read the dataset. Ensure iris.data file and this Jupyter notebook file at same location else provide the exact location. 

#Given dataset does not contain headers. Hence read the data with header as none option. 
df = pd.read_csv('iris.data', header=None)

#Define the headers based on information given for the data set. 
df.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class_Type']

#See first and last 5 rows of the data
print(df.head())
print (df.tail())

   Sepal_Length  Sepal_Width  Petal_Length  Petal_Width   Class_Type
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
     Sepal_Length  Sepal_Width  Petal_Length  Petal_Width      Class_Type
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica


In [37]:
#Get the information about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
Sepal_Length    150 non-null float64
Sepal_Width     150 non-null float64
Petal_Length    150 non-null float64
Petal_Width     150 non-null float64
Class_Type      150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


1. 4 independent features and a class type is given.
2. Total dataset has 150 rows. 
3. Features are of float type. 


In [38]:
df = pd.read_csv('iris.data', header=None)
df.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class_Type']

# To count hashable objects and creates a hash table. 
from collections import Counter

#Setting how floating-point errors are handled as we might divide few Expentancy value as 0 in the algorithm during calculations.
np.seterr(divide='ignore', invalid='ignore')

#Function to determine requested number of intervals from dataset using ChiMerge algorithm. 
def ChiMerge(dataframe, column_name, class_type, intervals_to_derive):
    
    #Chi-Merge works per feature and gives you best possible intervals. 
    #Print the size of dataset for which chi-merge is currently running
    print ("Data Set Length :", dataframe[column_name].count())
    
    #Remove the duplicates from feature column & target class
    unique_set = set(dataframe[column_name])
    unique_labels_set = set(dataframe[class_type])
 
    #Sort the data in ascending order
    uniq_list = sorted(unique_set)
    uniq_labels = sorted(unique_labels_set)
    
    print ("Sorted Unique Data:" ,uniq_list)
    print ("Sorted Unique Labels" ,uniq_labels )
    
    #Initiaze the intervals from unique list of values for each feature with zero steps
    curr_interval = [[uniq_list[i], uniq_list[i]] for i in range(len(uniq_list))] 
    
    #print ("Initial discretization with zero step \n", curr_interval)
         
    # Utility function for creating contigency matrix, will be used later
    Check_values = {l: 0 for l in uniq_labels} 
    
    #While loop to interate the algorithm till we reach to the count of intervals we need to derive. 
    while len(curr_interval) > intervals_to_derive:
        
        #Iniatiize the list to store calculated chi-square value
        chiSqVal = []
        
        #Run the loop for all possible intervals currently available. 
        for i in range(len(curr_interval)-1):
          
            #First Observed set of rows between current interval [lower : higher] value
            list_min = dataframe[dataframe[column_name].between(curr_interval[i][0], curr_interval[i][1])]
            
            #Second Observed set of rows between current interval [lower : higher] value
            list_max = dataframe[dataframe[column_name].between(curr_interval[i+1][0], curr_interval[i+1][1])]
            
            #Total number of rows in list_min & list_max would give grand gotal 
            grand_total = len(list_min) + len(list_max)
            
            #Identify class index for list_min for 3 given class Iris-setosa, Iris-versicolor and Iris-virginica
            #It calculates total number of samples belonging to each class type
            #E.g. 2 1 0 means 2 instances of Iris-setosa, 1 instance of Iris-versicolor and 0 instance of Iris-virginica
            
            #Samples from first observartion list_min
            #First row in contigency matrix
            list_min_classes = np.array([v for i, v in {**Check_values, **Counter(list_min[class_type])}.items()])
            
            #Samples from second observartion list_max 
            #second row in contigency matrix
            list_max_classes = np.array([v for i, v in {**Check_values, **Counter(list_max[class_type])}.items()])
            
            #Create the list with classes addition , column total in contigency matrix
            col_total = list_min_classes + list_max_classes
            
            #Calulate the Expentency Matrix from Contigency Matrix
            
            #First row of Expentency Matrix
            expectancy_row_one = col_total*sum(list_min_classes)/grand_total
            
            #Second row of Expentency Matrix
            expectancy_row_two = col_total*sum(list_max_classes)/grand_total
            
            #Find the ChiSquare using formula 
            # ChiSqaure = Sum of (Observed_val - Expected_val)**2/Expected Value 
            
            chiSquare = (list_min_classes - expectancy_row_one)**2/expectancy_row_one + (list_max_classes - expectancy_row_two)**2/expectancy_row_two
            
            #Convert NAN values to Zero 
            chiSquare = np.nan_to_num(chiSquare)
            
            # Add are the ChiSquares to determine ChiSquare value for running interval
            final_chiSquare = sum(chiSquare)
            
            # Append Final ChiSquare Value for comparison once running iteration gets over
            chiSqVal.append(final_chiSquare) 
        
        #End of loop for running iteration
          
        #As per chi-Square algorithm, identify the minimum chi-square values and Merge those intervals
        #Get the minimum Chi-Square Value
        min_chiSquare = min(chiSqVal) # Find the minimal Chi2 for current iteration
        
        #Get the index of the interval having minimum ChiSquare value
        for idx, itr in enumerate(chiSqVal):
            if itr == min_chiSquare:
                # The index with minimum chisquare value should be merged 
                min_chiSquare_idx = idx 
                break
        
        #Create the merged interval for second interval
        #Initialize the list to calculate merge intervals
        merged_intervals = [] 
        #2 flags used to control the loop and condition
        skip = False
        done = False
        
        #Traverse through all the intervals in current set
        for itr in range(len(curr_interval)):
            if skip:  # Got the first smallest interval and come out of loop
                skip = False 
                continue
                
            # IF interval matches the minimum value of all chi-squares obtained, merge adjacent intervals
            if itr == min_chiSquare_idx and not done: 
                #Add both the intervals into one set
                combined_intervals = curr_interval[itr] + curr_interval[itr+1]
                
                #Create the new interval with min of first interval amd max of second interval
                merged_intervals.append([min(combined_intervals), max(combined_intervals)])
                
                #Set the flags true
                skip = True
                done = True
            else:
                #Retain the interval for next iteration
                merged_intervals.append(curr_interval[itr])
        #Assign the new interval to current interval  for next cycle 
        curr_interval = merged_intervals    
    # End of While loop with desired number of features calculated using ChiMerge Algorithm

    #Print the results
    print ("Final ",intervals_to_derive," Chi-Squared Values are",sep='')
    for k in chiSqVal:
        print ('{:.4f}'.format(k),end="  ")
    
    #Print the discretized set of intervals for selected feature
    print("\nDesired intervals : ")
    for i in curr_interval:
        #Remove softspace to format it 
        print('{', i[0], ',', i[1], '}', sep='')
    print("\n")
      
       


 #LET THE STOPPING CRITERIA BE: MAX-INTERVAL 6
 #Set the Max interval as 6
max_intervals = 6

#Call the ChiMerge function for all features 
for attr in ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']:
    print('Feature Name    :', attr)
    ChiMerge(df, attr, 'Class_Type', max_intervals)

Feature Name    : Sepal_Length
Data Set Length : 150
Sorted Unique Data: [4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.6, 7.7, 7.9]
Sorted Unique Labels ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
Final 6 Chi-Squared Values are
5.8667  5.1724  21.2814  6.6770  5.0657  5.9376  
Desired intervals : 
{4.3,4.8}
{4.9,4.9}
{5.0,5.4}
{5.5,5.7}
{5.8,7.0}
{7.1,7.9}


Feature Name    : Sepal_Width
Data Set Length : 150
Sorted Unique Data: [2.0, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.4]
Sorted Unique Labels ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
Final 6 Chi-Squared Values are
2.3571  9.9821  5.7960  7.4125  19.0067  1.4400  
Desired intervals : 
{2.0,2.2}
{2.3,2.4}
{2.5,2.8}
{2.9,2.9}
{3.0,3.3}
{3.4,4.4}


Feature Name    : Petal_Length
Data Set Length : 150
Sorted Unique Data: [1.0, 1.1, 1.2

# SIMPLE ANALYSIS AND TEST RESULTS: SPLIT-POINTS, FINAL INTERVALS